In [1]:
import sys
sys.path.append('..')

import numpy as np
from pytenet.hartree_fock_mps import generate_single_state
from pytenet.operation import add_mps
from pytenet.hamiltonian_thc import eval_func, generate_thc_mpos_by_layer_qn, get_t, get_h1_spin, get_g_spin
from pytenet.global_krylov_method import generate_krylov_space_in_disk, get_W, get_S, generate_re_ortho_space_with_coeff, coeff_canonical_orthogonalization, remain_only_tridiagonal_elements
from pytenet.global_krylov_method import solve_ritz, generate_reduced_H_non_ortho, remain_only_tridiagonal_elements, coeff_gram_schmidt
import numpy as np
from scipy import sparse
import copy
import h5py
from numpy.linalg import norm
#np.set_printoptions(precision=4,suppress=True)
import scipy.io
import matplotlib.pyplot as plt
import pickle
import pytenet as ptn


Load and initialize datas: 

no is number of spatial orbitals

L is number of spinor orbitals, L = 2*no

t_spin is one-body integral in Chemist's notation (considering spins)

g_spin is two-body integral in Chemist's notation (considering spins)

X_mo and Z_mo are THC tensors, X_mo_up/down are X_mo considering spins

r_THC is THC rank

In [2]:
#load integrals
#with h5py.File("data_water/eri_water.hdf5", "r") as f:
with h5py.File("/work_fast/ge49cag/code_datas/hchain/NH_10/integral.hdf5", "r") as f:
    eri = f["eri"][()]
    hkin = f["hkin"][()]
    hnuc = f["hnuc"][()]

#print(np.linalg.norm(eri))
#print(eri.shape)

no = eri.shape[0]
MV = eri.reshape(no*no,no*no)

u = np.load("/work_fast/ge49cag/code_datas/hchain/NH_10/u.npy")
#u = np.load("/work_fast/ge49cag/pytenet_yu/water/x.npy")
X_mo = u.transpose(1,0)
g_thc, Z_mo = eval_func(u,eri,hkin,hnuc,)
h1 = hnuc+hkin
nmo = X_mo.shape[1]
L = 2*X_mo.shape[1]
g_thc = g_thc.reshape(nmo, nmo, nmo, nmo)
r_thc = X_mo.shape[0]

10
(10, 27)
(27, 27)
rl errV: 4.598965179239031e-05
abs errV: 0.00014647113920761016
errt: 0.0007926989428724057
errh: 0.00010479314802464695
errht: 0.00025419491299631246


These Hamiltonian are exact molecular Hamiltonian and molecular Hamiltonian reconstructed by THC tensors. The calculation cost time, so that we store them in disk and load them when needed. For water molecule H2O in STO-6G basis, the error is small for r_THC = 28.

Actually, considering there are always 10 electrons for a water molecule, we only retain the elements which operator quantum states with 10 electrons.

In [3]:
# #load Hamiltonian generated by above coefficients
#H_correct = scipy.io.mmread('/.mtx').tocsr()
# e1, v1 = sparse.linalg.eigsh(H_correct, which = 'SA', k = 1)
# e_ground = e1

# H_correct_10e = generate_Hamiltonian_with_occupation_number(H_correct.real, 10)

Generate THC-MPO by layers, using THC tensors. 
t_spin is used to create MPO for kinetic term.
It returns a list of H_mu_nu, each H_mu_nu is also a list, which contains four smaller MPOs with bond dims 2.
The final element of this list is MPO for kinetic term.

In [4]:
#generate thc_mpo
t = get_t(h1, eri)
H_mu_nu_list_spin_layer = generate_thc_mpos_by_layer_qn(X_mo, Z_mo, L, t)

print(type(H_mu_nu_list_spin_layer))
print(type(H_mu_nu_list_spin_layer[0]))
print(type(H_mu_nu_list_spin_layer[0][0]))
print((H_mu_nu_list_spin_layer[0][0].bond_dims))

<class 'list'>
<class 'list'>
<class 'pytenet.mpo.MPO'>
[1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1]


We can calculate elements in reduced Hamiltonian using conventional MPO.

Since we only need to store ONE block during contraction, memory needed is only $\mathcal{O}(L^2 M^2)$.

Create conventional mpo for molecular Hamiltonian:

In [5]:
g_phy =  eri.transpose(0, 2, 1, 3)
mpo_ref = ptn.hamiltonian.spin_molecular_hamiltonian_mpo(h1, g_phy)
print(mpo_ref.bond_dims)

e_ground = -12.407196480428968
e_1st_ex = -12.362467008874392

[1, 16, 70, 108, 162, 232, 162, 108, 70, 16, 1]


For ground state finding, we use Hatree fock state |11111111110000> as initial state.

For 1st excited state, please use single-excited Hatree-Fock state as initial state, or even superposition of several single-excited Hatree-Fock states as initial state.

In [6]:
initial = generate_single_state(10, [3, 3, 3, 3, 3, 0, 0, 0, 0, 0])
initial.orthonormalize('right')
#print(ptn.operation.operator_average(initial, mpo_ref) - e_ground)

1.0

We generate a group of orthogonal Krylov vectors using THC-MPO, with bond dim 40 for Krylov vectors. The vectors are stored in the folder = 'foldername', thus you don't have to generate them again for next time use. 

In [7]:
N_Krylov_1 = 30
psi_original_1 = copy.deepcopy(initial)
max_bond_Krylov_1 = 250
trunc_tol = 0
foldername_1 = f"/work_fast/ge49cag/code_datas/Krylov_H10"
#generate_krylov_space_in_disk(N_Krylov_1, H_mu_nu_list_spin_layer, psi_original_1, max_bond_Krylov_1, trunc_tol, r_thc, foldername_1)

Make use of method proposed in https://journals.aps.org/prb/abstract/10.1103/PhysRevB.85.205119 to improve the orthogonality of Krylov vectors. 

In [8]:
H_reduced_non_rotho_1 = generate_reduced_H_non_ortho(N_Krylov_1, foldername_1, mpo_ref)
coeff_1 = coeff_gram_schmidt(N_Krylov_1, foldername_1)
#H_reduced: elements calculated by post-orthogonalized Krylov vectos
H_reduced_1 = np.einsum('ik, kl, jl -> ij', coeff_1.conj(), H_reduced_non_rotho_1, coeff_1)


/work_fast/ge49cag/pytenet_thc_spin_cons/thc_experiments/../pytenet/global_krylov_method.py:214: ComplexWarning: Casting complex values to real discards the imaginary part
  H_reduced[i, j] = operator_inner_product(temp1, H_mpo, temp2)


/work_fast/ge49cag/pytenet_thc_spin_cons/thc_experiments/../pytenet/global_krylov_method.py:113: ComplexWarning: Casting complex values to real discards the imaginary part
  W[i,j] = vdot(temp1, temp2)


In [9]:
e_ritz_1, v_ritz_1 = solve_ritz(foldername_1, H_reduced_1, N_Krylov_1, coeff_1, max_bond_Krylov_1, e_ground, mpo_ref)
#if works, set N_krylov as 50 for this section, then use the 2nd one as initial state for excited state.

(4.9895638575465835+0j)
(2.9754418254647845+0j)
(1.8842586919638862+0j)
(1.118923977507352+0j)
(0.5891514410222225+0j)
(0.36575387369147805+0j)


In [13]:
N_Krylov_2 = 30
psi_original_2 = copy.deepcopy(v_ritz_1)
max_bond_Krylov_2 = 250
trunc_tol = 0
foldername_2= f"/work_fast/ge49cag/code_datas/Krylov_H10_restart"
generate_krylov_space_in_disk(N_Krylov_2, H_mu_nu_list_spin_layer, psi_original_2, max_bond_Krylov_2, trunc_tol, r_thc, foldername_2)

[1, 4, 16, 64, 233, 250, 233, 64, 16, 4, 1]
2
[1, 4, 16, 64, 235, 250, 235, 64, 16, 4, 1]
3
[1, 4, 16, 64, 235, 250, 235, 64, 16, 4, 1]
4
[1, 4, 16, 64, 235, 250, 235, 64, 16, 4, 1]
5
[1, 4, 16, 64, 235, 250, 235, 64, 16, 4, 1]
6
[1, 4, 16, 64, 240, 250, 240, 64, 16, 4, 1]
7
[1, 4, 16, 64, 241, 250, 241, 64, 16, 4, 1]
8
[1, 4, 16, 64, 244, 250, 244, 64, 16, 4, 1]
9
[1, 4, 16, 64, 245, 250, 245, 64, 16, 4, 1]
10
[1, 4, 16, 64, 245, 250, 245, 64, 16, 4, 1]
11
[1, 4, 16, 64, 250, 250, 250, 64, 16, 4, 1]
12
[1, 4, 16, 64, 248, 250, 249, 64, 16, 4, 1]
13
[1, 4, 16, 64, 248, 250, 249, 64, 16, 4, 1]
14
[1, 4, 16, 64, 248, 250, 249, 64, 16, 4, 1]
15
[1, 4, 16, 64, 248, 250, 249, 64, 16, 4, 1]
16
[1, 4, 16, 64, 248, 250, 249, 64, 16, 4, 1]
17
[1, 4, 16, 64, 250, 250, 250, 64, 16, 4, 1]
18
[1, 4, 16, 64, 250, 250, 250, 64, 16, 4, 1]
19
[1, 4, 16, 64, 250, 250, 250, 64, 16, 4, 1]
20
[1, 4, 16, 64, 250, 250, 250, 64, 16, 4, 1]
21
[1, 4, 16, 64, 250, 250, 250, 64, 16, 4, 1]
22
[1, 4, 16, 64, 250, 2

In [14]:
H_reduced_non_rotho_2 = generate_reduced_H_non_ortho(N_Krylov_2, foldername_2, mpo_ref)
coeff_2 = coeff_gram_schmidt(N_Krylov_2, foldername_2)
H_reduced_2 = np.einsum('ik, kl, jl -> ij', coeff_2.conj(), H_reduced_non_rotho_2, coeff_2)


/work_fast/ge49cag/pytenet_thc_spin_cons/thc_experiments/../pytenet/global_krylov_method.py:214: ComplexWarning: Casting complex values to real discards the imaginary part
  H_reduced[i, j] = operator_inner_product(temp1, H_mpo, temp2)
/work_fast/ge49cag/pytenet_thc_spin_cons/thc_experiments/../pytenet/global_krylov_method.py:113: ComplexWarning: Casting complex values to real discards the imaginary part
  W[i,j] = vdot(temp1, temp2)


In [15]:
e_ritz_2, v_ritz_2 = solve_ritz(foldername_2, H_reduced_2, N_Krylov_2, coeff_2, max_bond_Krylov_2, e_ground, mpo_ref)

(0.06698301097829074+0j)
(0.0026002560049107615+0j)
(0.0003776699617112911+0j)
(3.4391775949771386e-05+0j)
(1.728864507200001e-05+0j)
(1.5936947953321123e-05+0j)


restart:

In [17]:
N_Krylov_3 = 20
psi_original_3 = copy.deepcopy(v_ritz_2)
max_bond_Krylov_3 = 250
trunc_tol = 0
foldername_3 = f"/work_fast/ge49cag/code_datas/Krylov_H10_restart_2"
generate_krylov_space_in_disk(N_Krylov_3, H_mu_nu_list_spin_layer, psi_original_3, max_bond_Krylov_3, trunc_tol, r_thc, foldername_3)


[1, 4, 16, 64, 250, 250, 250, 64, 16, 4, 1]
2
[1, 4, 16, 64, 250, 250, 250, 64, 16, 4, 1]
3
[1, 4, 16, 64, 250, 250, 250, 64, 16, 4, 1]
4
[1, 4, 16, 64, 250, 250, 250, 64, 16, 4, 1]
5
[1, 4, 16, 64, 250, 250, 250, 64, 16, 4, 1]
6
[1, 4, 16, 64, 250, 250, 250, 64, 16, 4, 1]
7
[1, 4, 16, 64, 250, 250, 250, 64, 16, 4, 1]
8
[1, 4, 16, 64, 250, 250, 250, 64, 16, 4, 1]
9
[1, 4, 16, 64, 250, 250, 250, 64, 16, 4, 1]
10
[1, 4, 16, 64, 250, 250, 250, 64, 16, 4, 1]
11
[1, 4, 16, 64, 250, 250, 250, 64, 16, 4, 1]
12
[1, 4, 16, 64, 250, 250, 250, 64, 16, 4, 1]
13
[1, 4, 16, 64, 250, 250, 250, 64, 16, 4, 1]
14
[1, 4, 16, 64, 250, 250, 250, 64, 16, 4, 1]
15
[1, 4, 16, 64, 250, 250, 250, 64, 16, 4, 1]
16
[1, 4, 16, 64, 250, 250, 250, 64, 16, 4, 1]
17
[1, 4, 16, 64, 250, 250, 250, 64, 16, 4, 1]
18
[1, 4, 16, 64, 250, 250, 250, 64, 16, 4, 1]
19
[1, 4, 16, 64, 250, 250, 250, 64, 16, 4, 1]


In [18]:
H_reduced_non_rotho_3 = generate_reduced_H_non_ortho(N_Krylov_3, foldername_3, mpo_ref)
coeff_3 = coeff_gram_schmidt(N_Krylov_3, foldername_3)
H_reduced_3 = np.einsum('ik, kl, jl -> ij', coeff_3.conj(), H_reduced_non_rotho_3, coeff_3)

In [19]:
e_ritz_3, v_ritz_3 = solve_ritz(foldername_3, H_reduced_3, N_Krylov_3, coeff_3, max_bond_Krylov_3, e_ground, mpo_ref)

(3.2456567744532094e-06+0j)
(3.0369881329761483e-06+0j)
(3.032986882089972e-06+0j)
(3.0320862745014665e-06+0j)


restart:

In [20]:
N_Krylov_4 = 20
psi_original_4 = copy.deepcopy(v_ritz_3)
max_bond_Krylov_4 = 250
trunc_tol = 0
foldername_4 = f"/work_fast/ge49cag/code_datas/Krylov_H10_3"
generate_krylov_space_in_disk(N_Krylov_4, H_mu_nu_list_spin_layer, psi_original_4, max_bond_Krylov_4, trunc_tol, r_thc, foldername_4)


[1, 4, 16, 64, 250, 250, 250, 64, 16, 4, 1]
2
[1, 4, 16, 64, 250, 250, 250, 64, 16, 4, 1]
3
[1, 4, 16, 64, 250, 250, 250, 64, 16, 4, 1]
4
[1, 4, 16, 64, 250, 250, 250, 64, 16, 4, 1]
5
[1, 4, 16, 64, 249, 250, 250, 64, 16, 4, 1]
6
[1, 4, 16, 64, 249, 250, 250, 64, 16, 4, 1]
7
[1, 4, 16, 64, 249, 250, 250, 64, 16, 4, 1]
8
[1, 4, 16, 64, 249, 250, 250, 64, 16, 4, 1]
9
[1, 4, 16, 64, 249, 250, 250, 64, 16, 4, 1]
10
[1, 4, 16, 64, 249, 250, 250, 64, 16, 4, 1]
11
[1, 4, 16, 64, 248, 250, 249, 64, 16, 4, 1]
12
[1, 4, 16, 64, 249, 250, 250, 64, 16, 4, 1]
13
[1, 4, 16, 64, 249, 250, 250, 64, 16, 4, 1]
14
[1, 4, 16, 64, 249, 250, 249, 64, 16, 4, 1]
15
[1, 4, 16, 64, 249, 250, 250, 64, 16, 4, 1]
16
[1, 4, 16, 64, 249, 250, 250, 64, 16, 4, 1]
17
[1, 4, 16, 64, 249, 250, 249, 64, 16, 4, 1]
18
[1, 4, 16, 64, 250, 250, 250, 64, 16, 4, 1]
19
[1, 4, 16, 64, 250, 250, 250, 64, 16, 4, 1]


In [21]:
H_reduced_non_rotho_4 = generate_reduced_H_non_ortho(N_Krylov_4, foldername_4, mpo_ref)
coeff_4 = coeff_gram_schmidt(N_Krylov_4, foldername_4)
H_reduced_4 = np.einsum('ik, kl, jl -> ij', coeff_4.conj(), H_reduced_non_rotho_4, coeff_4)


In [22]:
e_ritz_4, v_ritz_4 = solve_ritz(foldername_4, H_reduced_4, N_Krylov_4, coeff_4, max_bond_Krylov_4, e_ground, mpo_ref)

(9.319979259458933e-07+0j)
(9.30569894919131e-07+0j)
(9.302608212635732e-07+0j)
(9.297268608321474e-07+0j)


In [24]:
e_final, v_final = np.linalg.eigh(H_reduced_4)
print(e_final[0] - e_ground)

9.360629924515251e-07


In [29]:
# filename = f"/work_fast/ge49cag/code_datas" + f"/H10_ground_state.pkl"
# with open(filename, 'wb') as file:
#     pickle.dump(v_ritz_4, file)

filename = f"/work_fast/ge49cag/code_datas" + f"/H10_ground_state.pkl"
with open(filename, 'rb') as file:
    H10_ground = pickle.load(file)

ptn.operation.operator_average(H10_ground, mpo_ref) - e_ground

(9.297268608321474e-07+0j)

In [31]:
error_list = [
    4.9895638575465835,
    2.9754418254647845,
    1.8842586919638862,
    1.118923977507352,
    0.5891514410222225,
    0.36575387369147805,
    0.06698301097829074,
    0.0026002560049107615,
    0.0003776699617112911,
    3.4391775949771386e-05,
    1.728864507200001e-05,
    1.5936947953321123e-05,
    3.2456567744532094e-06,
    3.0369881329761483e-06,
    3.032986882089972e-06,
    3.0320862745014665e-06,
    9.319979259458933e-07,
    9.30569894919131e-07,
    9.302608212635732e-07,
    9.297268608321474e-07
]


In [32]:
error_list_divided_by_10 = [error / 10 for error in error_list]
print(error_list_divided_by_10)

[0.49895638575465834, 0.29754418254647846, 0.18842586919638862, 0.1118923977507352, 0.05891514410222225, 0.036575387369147803, 0.006698301097829073, 0.0002600256004910761, 3.776699617112911e-05, 3.4391775949771388e-06, 1.7288645072000008e-06, 1.5936947953321123e-06, 3.2456567744532094e-07, 3.0369881329761485e-07, 3.032986882089972e-07, 3.0320862745014663e-07, 9.319979259458932e-08, 9.30569894919131e-08, 9.302608212635733e-08, 9.297268608321473e-08]
